In [1]:
# 스파크 드라이버와 워커의 파이썬 버전이 다를 경우, 
# PYSPARK_PYTHON 와 PYSPARK_DRIVER_PYTHON 를 환경변수 지정 
# 예시 : .zshrc 파일에 아래 코드 입력
#           export PYSPARK_PYTHON=$HOME/.pyenv/versions/3.12.1/bin/python3.12
#           export PYSPARK_DRIVER_PYTHON=$HOME/.pyenv/versions/3.12.1/bin/python3.12
# 위의 설정을 해도 오류가 날 경우, 스크립트에 아래 코드 입력
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
# 스파크 세션 생성
import pyspark
from pyspark.sql import SparkSession

# SynapseML : 마이크로소프트에서 만든 스파크용 ML 라이브러리
# https://microsoft.github.io/SynapseML/
# config에서 synapseml 모듈 불러오기
spark = SparkSession.builder \
    .appName("Spark study - 231219") \
    .config("spark.jars.packages", "com.microsoft.azure:synapseml_2.12:1.0.2") \
    .config("spark.jars.repositories", "https://mmlspark.azureedge.net/maven") \
    .getOrCreate()

23/12/19 16:48:43 WARN Utils: Your hostname, KJH-DESKTOP resolves to a loopback address: 127.0.1.1; using 192.168.69.220 instead (on interface eth0)
23/12/19 16:48:43 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
https://mmlspark.azureedge.net/maven added as a remote repository with the name: repo-1
Ivy Default Cache set to: /home/kjh/.ivy2/cache
The jars for the packages stored in: /home/kjh/.ivy2/jars
com.microsoft.azure#synapseml_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-ce897387-11f7-4c2a-b3a9-42fdc952be35;1.0
	confs: [default]
	found com.microsoft.azure#synapseml_2.12;1.0.2 in central
	found com.microsoft.azure#synapseml-core_2.12;1.0.2 in central
	found org.apache.spark#spark-avro_2.12;3.4.1 in central
	found org.tukaani#xz;1.9 in central
	found commons-lang#commons-lang;2.6 in central
	found org.scalactic#scalactic_2.12;3.2.14 in central
	found org.scala-lang#scala-reflect;2.12.15 in central
	found io.spr

:: loading settings :: url = jar:file:/home/kjh/app/spark/3.5.0/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.httpcomponents.client5#httpclient5;5.1.3 in central
	found org.apache.httpcomponents.core5#httpcore5;5.1.3 in central
	found org.apache.httpcomponents.core5#httpcore5-h2;5.1.3 in central
	found org.slf4j#slf4j-api;1.7.25 in central
	found commons-codec#commons-codec;1.15 in central
	found org.apache.httpcomponents#httpmime;4.5.13 in central
	found org.apache.httpcomponents#httpclient;4.5.13 in central
	found org.apache.httpcomponents#httpcore;4.4.13 in central
	found commons-logging#commons-logging;1.2 in central
	found com.linkedin.isolation-forest#isolation-forest_3.4.1_2.12;3.0.3 in central
	found com.chuusai#shapeless_2.12;2.3.10 in central
	found org.testng#testng;6.8.8 in central
	found org.beanshell#bsh;2.0b4 in central
	found com.beust#jcommander;1.27 in central
	found org.scalanlp#breeze_2.12;2.1.0 in central
	found org.scalanlp#breeze-macros_2.12;2.1.0 in central
	found org.typelevel#spire_2.12;0.17.0 in central
	found org.typelevel#spire-macros_2.12;0.17.0 

In [3]:
import pyarrow.parquet as pq

parquet_file = pq.read_table("file:///home/kjh/data/taxi/yellow_tripdata_2022-01.parquet")
print(parquet_file.schema)

VendorID: int64
tpep_pickup_datetime: timestamp[us]
tpep_dropoff_datetime: timestamp[us]
passenger_count: double
trip_distance: double
RatecodeID: double
store_and_fwd_flag: string
PULocationID: int64
DOLocationID: int64
payment_type: int64
fare_amount: double
extra: double
mta_tax: double
tip_amount: double
tolls_amount: double
improvement_surcharge: double
total_amount: double
congestion_surcharge: double
airport_fee: double
-- schema metadata --
pandas: '{"index_columns": [], "column_indexes": [], "columns": [{"name":' + 2492


In [4]:
# Parquet 파일이 있는 폴더 경로
parquet_folder_path = "/home/kjh/data/taxi"

# 폴더 내의 모든 Parquet 파일을 리스트로 가져옴
parquet_files = [file for file in os.listdir(parquet_folder_path) if file.endswith('.parquet')]

In [5]:
# 각 Parquet 파일을 따로 DataFrame으로 읽음
dataframes = []
for file in parquet_files:
    file_path = os.path.join(parquet_folder_path, file)
    df = spark.read.parquet(file_path)
    dataframes.append(df)

In [6]:
# 여러 데이터프레임을 합치는 과정

from functools import reduce
from pyspark.sql import DataFrame

# 두 DataFrame을 합치는 함수
def union_two_dataframes(df1, df2):
    return df1.union(df2)

# 모든 DataFrame을 하나로 합치기
df_raw = reduce(union_two_dataframes, dataframes)

In [7]:
df_raw.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [8]:
df_raw.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2022-07-01 00:20:06|  2022-07-01 00:39:13|            1.0|         10.1|       1.0|                 N|          70|          33|           1|       28.5|  0.5|    0.5|       8.

In [9]:
df_raw.printSchema()

root
 |-- VendorID: long (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)



In [10]:
df_raw.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2022-07-01 00:20:06|  2022-07-01 00:39:13|            1.0|         10.1|       1.0|                 N|          70|          33|           1|       28.5|  0.5|    0.5|       8.

In [11]:
df_drop = df_raw.drop('VendorID', 'RatecodeID', 'store_and_fwd_flag', 'PULocationID', 'DOLocationID', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'tolls_amount', 'improvement_surcharge')

In [12]:
df_drop.show()

+--------------------+---------------------+---------------+-------------+------------+------------+--------------------+-----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|payment_type|total_amount|congestion_surcharge|airport_fee|
+--------------------+---------------------+---------------+-------------+------------+------------+--------------------+-----------+
| 2022-07-01 00:20:06|  2022-07-01 00:39:13|            1.0|         10.1|           1|        38.7|                 0.0|        0.0|
| 2022-07-01 00:29:11|  2022-07-01 00:38:00|            1.0|         1.67|           1|        12.8|                 2.5|        0.0|
| 2022-07-01 00:03:56|  2022-07-01 00:11:49|            1.0|          0.9|           1|        11.8|                 2.5|        0.0|
| 2022-07-01 00:18:36|  2022-07-01 00:52:44|            1.0|         14.8|           1|        65.3|                 0.0|        0.0|
| 2022-07-01 00:15:50|  2022-07-01 00:22:21|            1.0|  

In [13]:
df_drop.createOrReplaceTempView("temp_df")

In [14]:
spark.sql("""
          DESCRIBE temp_df
          """).show()

+--------------------+-------------+-------+
|            col_name|    data_type|comment|
+--------------------+-------------+-------+
|tpep_pickup_datetime|timestamp_ntz|   NULL|
|tpep_dropoff_date...|timestamp_ntz|   NULL|
|     passenger_count|       double|   NULL|
|       trip_distance|       double|   NULL|
|        payment_type|       bigint|   NULL|
|        total_amount|       double|   NULL|
|congestion_surcharge|       double|   NULL|
|         airport_fee|       double|   NULL|
+--------------------+-------------+-------+



In [15]:
spark.sql("""
          SELECT COUNT(*) CNT
          FROM temp_df
          """).show()

+--------+
|     CNT|
+--------+
|59149718|
+--------+



In [16]:
spark.sql("""
          SELECT *
          FROM temp_df
          ORDER BY tpep_pickup_datetime
          """).show()

+--------------------+---------------------+---------------+-------------+------------+------------+--------------------+-----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|payment_type|total_amount|congestion_surcharge|airport_fee|
+--------------------+---------------------+---------------+-------------+------------+------------+--------------------+-----------+
| 2001-01-01 00:03:14|  2001-01-01 01:12:47|            1.0|        20.77|           2|       61.85|                 2.5|        0.0|
| 2001-01-01 00:06:49|  2001-01-01 15:42:11|            2.0|         7.41|           1|        46.9|                 2.5|        0.0|
| 2001-01-01 00:07:36|  2001-01-01 00:16:31|            1.0|         1.68|           2|        14.7|                 2.5|        0.0|
| 2001-01-01 00:08:31|  2001-01-01 16:01:27|            1.0|        18.63|           1|       98.76|                 2.5|       1.75|
| 2001-01-01 00:08:42|  2001-01-01 14:13:51|            1.0|  

In [17]:
df_drop = spark.sql("""
                    SELECT *
                    FROM temp_df
                    WHERE tpep_pickup_datetime >= '2022-01-01 00:00:00'
                        AND tpep_pickup_datetime < '2023-07-01 00:00:00'
                    """)

In [18]:
df_drop.createOrReplaceTempView('temp_df')

In [19]:
# 결측치 확인

from pyspark.sql.functions import col, count, when, isnull

# 각 컬럼에 대해 non-null 값을 계산
null_counts = df_drop.select([count(when(isnull(c), c)).alias(c) for c in df_drop.columns])

null_counts.show()

+--------------------+---------------------+---------------+-------------+------------+------------+--------------------+-----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|payment_type|total_amount|congestion_surcharge|airport_fee|
+--------------------+---------------------+---------------+-------------+------------+------------+--------------------+-----------+
|                   0|                    0|        1896855|            0|           0|           0|             1896855|    1896855|
+--------------------+---------------------+---------------+-------------+------------+------------+--------------------+-----------+



In [20]:
columns = ['passenger_count', 'trip_distance', 'payment_type', 'total_amount', 'congestion_surcharge', 'airport_fee']
# 각 컬럼에 대해 음수 값을 계산
negative_counts = df_drop.select([count(when(col(c) < 0, c)).alias(c) for c in columns])

negative_counts.show()

+---------------+-------------+------------+------------+--------------------+-----------+
|passenger_count|trip_distance|payment_type|total_amount|congestion_surcharge|airport_fee|
+---------------+-------------+------------+------------+--------------------+-----------+
|              0|            0|           0|      428114|              341599|      49879|
+---------------+-------------+------------+------------+--------------------+-----------+



In [21]:
spark.sql("""
          SELECT *
          FROM temp_df
          ORDER BY tpep_pickup_datetime
          """).show()

+--------------------+---------------------+---------------+-------------+------------+------------+--------------------+-----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|payment_type|total_amount|congestion_surcharge|airport_fee|
+--------------------+---------------------+---------------+-------------+------------+------------+--------------------+-----------+
| 2022-01-01 00:00:08|  2022-01-01 00:14:14|            1.0|         7.94|           1|       30.41|                 0.0|       1.25|
| 2022-01-01 00:00:11|  2022-01-01 00:14:29|            1.0|         5.55|           2|       20.05|                 0.0|       1.25|
| 2022-01-01 00:00:17|  2022-01-01 00:31:04|            1.0|         3.86|           1|       28.56|                 2.5|        0.0|
| 2022-01-01 00:00:18|  2022-01-01 00:13:27|            1.0|         3.37|           1|       20.16|                 2.5|        0.0|
| 2022-01-01 00:00:20|  2022-01-01 00:20:51|            1.0|  

In [22]:
spark.sql("""
          SELECT *
          FROM temp_df
          WHERE airport_fee < 0
          ORDER BY tpep_pickup_datetime
          """).show()

+--------------------+---------------------+---------------+-------------+------------+------------+--------------------+-----------+
|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|payment_type|total_amount|congestion_surcharge|airport_fee|
+--------------------+---------------------+---------------+-------------+------------+------------+--------------------+-----------+
| 2022-01-01 00:08:41|  2022-01-01 00:28:34|            1.0|         7.97|           4|      -26.55|                 0.0|      -1.25|
| 2022-01-01 00:26:37|  2022-01-01 00:39:35|            1.0|         7.38|           4|      -23.55|                 0.0|      -1.25|
| 2022-01-01 00:29:38|  2022-01-01 00:34:30|            1.0|         0.57|           3|      -10.05|                -2.5|      -1.25|
| 2022-01-01 00:43:44|  2022-01-01 01:14:51|            1.0|        22.71|           2|      -63.05|                 0.0|      -1.25|
| 2022-01-01 00:44:44|  2022-01-01 00:44:48|            2.0|  

In [23]:
df_drop = spark.sql("""
                    SELECT *
                    FROM temp_df
                    WHERE passenger_count >= 1
                    """)

In [24]:
df_drop.createOrReplaceTempView('temp_df')

In [25]:
spark.sql("""
          SELECT count(*) cnt
          FROM temp_df
          """).show()

+--------+
|     cnt|
+--------+
|56161187|
+--------+



In [26]:
df_month = spark.sql("""
                     SELECT 
                        DATE_FORMAT(tpep_pickup_datetime, 'yyyy-MM') AS tpep_pickup_datetime_year, 
                        *
                     FROM temp_df
                     """)

In [27]:
df_month.show()

+-------------------------+--------------------+---------------------+---------------+-------------+------------+------------+--------------------+-----------+
|tpep_pickup_datetime_year|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|payment_type|total_amount|congestion_surcharge|airport_fee|
+-------------------------+--------------------+---------------------+---------------+-------------+------------+------------+--------------------+-----------+
|                  2022-07| 2022-07-01 00:20:06|  2022-07-01 00:39:13|            1.0|         10.1|           1|        38.7|                 0.0|        0.0|
|                  2022-07| 2022-07-01 00:29:11|  2022-07-01 00:38:00|            1.0|         1.67|           1|        12.8|                 2.5|        0.0|
|                  2022-07| 2022-07-01 00:03:56|  2022-07-01 00:11:49|            1.0|          0.9|           1|        11.8|                 2.5|        0.0|
|                  2022-07| 2022-07-01 0

In [28]:
df_month.createOrReplaceTempView('temp_df_month')

In [29]:
df_final = spark.sql("""
                     SELECT 
                        tpep_pickup_datetime_year AS year_month,
                        SUM(passenger_count) AS passenger_count
                     FROM temp_df_month
                     GROUP BY tpep_pickup_datetime_year
                     ORDER BY tpep_pickup_datetime_year
                     """)

In [30]:
df_final.show()

+----------+---------------+
|year_month|passenger_count|
+----------+---------------+
|   2022-01|      3324082.0|
|   2022-02|      4006108.0|
|   2022-03|      4875580.0|
|   2022-04|      4921688.0|
|   2022-05|      4821239.0|
|   2022-06|      4792144.0|
|   2022-07|      4386947.0|
|   2022-08|      4353905.0|
|   2022-09|      4252079.0|
|   2022-10|      4905091.0|
|   2022-11|      4360904.0|
|   2022-12|      4646436.0|
|   2023-01|      4080782.0|
|   2023-02|      3838267.0|
|   2023-03|      4481350.0|
|   2023-04|      4421675.0|
|   2023-05|      4636051.0|
|   2023-06|      4390868.0|
+----------+---------------+



In [31]:
# 월별 택시 이용객 수 시각화하기

import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
from datetime import datetime


pandas_df = df_final.toPandas()

# Jupyter 노트북 내에서 그래프를 출력하기 위한 설정
output_notebook()

# 날짜 형식 변환 (yyyy-MM 문자열을 datetime 객체로 변환)
pandas_df['year_month'] = pd.to_datetime(pandas_df['year_month'])

# 데이터 소스 생성
source = ColumnDataSource(pandas_df)

# 그래프 생성
p = figure(x_axis_type="datetime", title="월별 승객 수")
p.line(x='year_month', y='passenger_count', source=source)

# 축 이름 설정
p.xaxis.axis_label = "년월"
p.yaxis.axis_label = "승객 수"

# 그래프 출력
show(p)

Loading BokehJS ...

In [32]:
pandas_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18 entries, 0 to 17
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   year_month       18 non-null     datetime64[ns]
 1   passenger_count  18 non-null     float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 420.0 bytes
